# Is it a Magic, Pokèmon, or Yugioh! card?
Exercise for Lecture 1 of the course [Practucal Deep Learning for Coders](https://course.fast.ai/Lessons/lesson1.html) by [NerusSkyhigh](https://guglielmogrillo.com/).

My aim is to train the network to distinguish between [Magic: The Gathering](https://magic.wizards.com/en/articles/archive/anatomy-magic-card-2006-10-21), [Yugioh!](https://outof.cards/yu-gi-oh/guides/146-understanding-how-yu-gi-oh-cards-are-laid-out-card-anatomy-101/), and [Pokémon TCG](https://images2.fanpop.com/image/photos/9100000/Card-Anatomy-pokemon-trading-cards-9164776-725-740.jpg) cards.

At first, I tried to use the [duckduckgo](https://duckduckgo.com/) search feature already available in the [Is it a bird?](https://www.kaggle.com/code/jhoward/is-it-a-bird-creating-a-model-from-your-own-data) notebook, but the data was very noisy and not always related to the task. I decided to overcome this problem by looking for high-quality scans/images of the cards. A lot of official and community-driven databases are available, I decided to work with the ones that didn't require any account/API key so that I could share a self-consistent notebook. This choice made the training a lot simpler at the expense of the accuracy on non high-qualities images. I'm still looking for a way to "artificially" add some noise to simulate the presence of nonuniform light in photos.

In [1]:
# This is not needed on the first run, but allows for a fresh
# run after testing. Otherwise the number of images used
# keep increasing
#!rm -rf *

# Check internet connectivity and install libraries
import socket,warnings
try:
    socket.setdefaulttimeout(1)
    socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect(('1.1.1.1', 53))
    print("Internet OK")
except socket.error as ex: raise Exception("STOP: No internet. Click '>|' in top right and set 'Internet' switch to on")


import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -Uqq fastai

    print("Dependencies installed on kaggle")

## Step 1: Download images

In [2]:
from fastdownload import download_url
from fastai.vision.all import *

N_images = 100
path = Path('cards')

### Pokémon Cards

Pokémon cards will be downloaded from [Pokemon-TCG](https://github.com/PokemonTCG/pokemon-tcg-data) using the link on their GitHub repository.

I set a 0.05 seconds delay (=20 requests/sec) so that I won't overload the server. Only the Yugioh!'s database specified a cap on maximum requests, but I decided to extend that advice to all the databases.

In [3]:
!git clone https://github.com/PokemonTCG/pokemon-tcg-data/ 2>/dev/null
#!ls -al pokemon-tcg-data/cards/en | tail -6

import os, json, random
from time import sleep

def get_random_pokemon_name_image_url():
    sets = os.listdir("pokemon-tcg-data/cards/en/")
    set_name = random.choice(sets);
    
    with open('pokemon-tcg-data/cards/en/'+set_name, 'r') as f:
        pokemon_set = json.load(f)
        pokemon_card = random.choice(pokemon_set)
        
        return pokemon_card['name'], pokemon_card['images']['small']

name, url = get_random_pokemon_name_image_url()
name = ''.join(ch for ch in name if ch.isalnum())+".png"
print(name, url)
download_url(url, name, show_progress=False)
Image.open(name).to_thumb(256,256)

In [4]:
dest = (path/'pokemon')
dest.mkdir(exist_ok=True, parents=True)

for i in range(N_images):
    name, url = get_random_pokemon_name_image_url()
    name = ''.join(ch for ch in name if ch.isalnum())+".png"
    try:
        download_url(url, dest/name, show_progress=False)
    except:
        # It seems that some links provided are broken and
        # return an error + the back of a Pokémon card
        # I don't want to look into it, I'll just try again
        # with a different one
        print("Failed to download", name, url)
        i = i-1 # Try again with a different one

    sleep(0.05)

resize_images(path/"pokemon", max_size=300, dest=path/"pokemon")

### Yugioh cards
Yugioh images will be downloaded from [db.ygoprodeck.com](https://db.ygoprodeck.com/api-guide/)'s api.

In [6]:
import requests, json
from time import sleep

def get_random_yugioh_name_image_url():
    url = "https://db.ygoprodeck.com/api/v7/randomcard.php"
    # This header is required to use randomcard otherwise the server respondes with
    # an _Error 403 - Forbidden_.
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    result = requests.get(url, headers=headers)
    card = json.loads(result.content.decode())  
    
    return card['name']+".jpg", card['card_images'][0]['image_url']

name, url = get_random_yugioh_name_image_url()
name = ''.join(ch for ch in name if ch.isalnum())+".jpg"
print(name, url)
download_url(url, name, show_progress=False)
Image.open(name).to_thumb(256,256)

In [7]:
dest = (path/'yugioh')
dest.mkdir(exist_ok=True, parents=True)

for i in range(N_images):
    name, url = get_random_yugioh_name_image_url()
    name = ''.join(ch for ch in name if ch.isalnum())+".jpg"
    download_url(url, dest/name, show_progress=False)
    sleep(0.05)

resize_images(path/'yugioh', max_size=300, dest=path/'yugioh')

### Magic cards
Magic cards will be downloaded from scryfall using [mtgjson](https://mtgjson.com/)'s link.

In [9]:
!wget -nc https://mtgjson.com/api/v5/AllPrintings.json
    
with open('AllPrintings.json', 'r') as f:
        magic_json = json.load(f)

def get_random_magic_name_image_url():
        magic_set = random.choice( list(magic_json['data'].values()) )
        
        while(not ("name" in magic_set.keys()) or (len(magic_set['cards']) == 0)  ):
            magic_set = random.choice( list(magic_json['data'].values()) )

        name = magic_set['cards'][0]['name']
        scryfallId = magic_set['cards'][0]['identifiers']['scryfallId']
        
        url = "https://api.scryfall.com/cards/${scryfallId}?format=image".replace("${scryfallId}", scryfallId)
        return name+".jpg", url
    
    
name, url = get_random_magic_name_image_url()
name = ''.join(ch for ch in name if ch.isalnum())+".jpg"
print(name, url)
download_url(url, name, show_progress=False)
Image.open(name).to_thumb(256,256)

In [10]:
dest = (path/'magic')
dest.mkdir(exist_ok=True, parents=True)

for i in range(N_images):
    name, url = get_random_magic_name_image_url()
    name = ''.join(ch for ch in name if ch.isalnum())+".jpg"
    download_url(url, dest/name, show_progress=False)
    sleep(0.05)
    
resize_images(path/'magic', max_size=300, dest=path/'magic')

## Step 2: Train our model


To train a model, we'll need `DataLoaders`, which is an object that contains a *training set* (the images used to create a model) and a *validation set* (the images used to check the accuracy of a model -- not used during training). In `fastai` we can create that easily using a `DataBlock`, and view sample images from it:

In [12]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    # As I download random images the seed is not really needed.
    # I keep 42 just for the memes.
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, verbose=True)

dls.show_batch(max_n=16)
dls.vocab

In [13]:
learn = vision_learner(dls, resnet18, metrics=error_rate, pretrained=True)
learn.fine_tune(3)

## Step 3: Use our model (and build your own!)

I'm now going to test the model with some images from internet:

#### Blue eyes white dragon, Ponyta and Bogstomper
<img src="https://m.media-amazon.com/images/I/51gCu4PGv4L._AC_.jpg" style="float:left;height:200px" />
<img src="https://i.ebayimg.com/images/g/tf0AAOSwvaJgKtXj/s-l500.jpg" style="float:left;height:200px" />
<img src="https://static.cardmarket.com/img/de13b4405280acbbad6a9d84f63e271d/items/1/XM20/381303.jpg" style="float:left;height:200px" /></br>

&nbsp;&nbsp; Test cards are correctly classified with high probabilities.

#### Meme cards  
<p>
<img src="https://www.memesmonkey.com/images/memesmonkey/09/09407be1736c6aba9c92225f1065e00c.jpeg" style="float:left;height:150px" />
<img src="https://keepmeme.com/meme/20211103/trap-card-bitch-hold-on.jpg" style="float:left;height:150px"x />
<img src="https://i.kym-cdn.com/photos/images/facebook/001/118/677/cd6.jpg" style="float:left;height:150px" /> </div>

&nbsp;&nbsp; Meme cards are still correctly classified with high probabilities.

#### Mosaic, Bird and Gay pride flag
<img src="https://i0.wp.com/tamoravenna.info/wp-content/uploads/2017/01/Ancient_Roman_Season_Mosaic-4.jpg?fit=1200%2C1200&ssl=1" style="float:left;height:150px" />
<img src="https://www.ouinolanguages.com/assets/Italian/vocab/17/images/pic2.jpg" style="float:left;height:150px" />
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/48/Gay_Pride_Flag.svg/800px-Gay_Pride_Flag.svg.png" style="float:left;height:150px" />

&nbsp;&nbsp; On random images the network shows an interesting results: it seems like that images with a lot of variety in colors are classified as Pokémon images. That seems fair as Pokémon images tend to be more flashy than Magic or Yugioh ones.


In [24]:
yugioh_card = "https://m.media-amazon.com/images/I/51gCu4PGv4L._AC_.jpg"
pokemon_card = "https://i.ebayimg.com/images/g/tf0AAOSwvaJgKtXj/s-l500.jpg"
magic_card = "https://static.cardmarket.com/img/de13b4405280acbbad6a9d84f63e271d/items/1/XM20/381303.jpg"

mosaic_url = "https://i0.wp.com/tamoravenna.info/wp-content/uploads/2017/01/Ancient_Roman_Season_Mosaic-4.jpg?fit=1200%2C1200&ssl=1"
bird_url = "https://www.ouinolanguages.com/assets/Italian/vocab/17/images/pic2.jpg"
flag_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/48/Gay_Pride_Flag.svg/800px-Gay_Pride_Flag.svg.png"

meme_magic_card = "https://www.memesmonkey.com/images/memesmonkey/09/09407be1736c6aba9c92225f1065e00c.jpeg"
meme_yugioh_card = "https://keepmeme.com/meme/20211103/trap-card-bitch-hold-on.jpg"
meme_pokemon_card = "https://i.kym-cdn.com/photos/images/facebook/001/118/677/cd6.jpg"


download_url(yugioh_card, 'test.jpg', show_progress=False)

game,_,probs = learn.predict(PILImage.create('test.jpg'))
print(f"This is a {game} card.\n")

print(f"Probability it's a Magic card: {probs[0]:.4f}")
print(f"Probability it's a Pokémon card: {probs[1]:.4f}")
print(f"Probability it's a Yugioh card: {probs[2]:.4f}")
print("")
Image.open('test.jpg').to_thumb(256,256)